In [1]:
import os
import glob
import pickle
import ast
import json
import numpy as np
import seaborn as sns
import pandas as pd
from dance_evaluation import *
import matplotlib.pyplot as plt

### Analyze Hits from multiple sensors

In [2]:
config1 = {"a": 60, "b": [100,110,120,130,140,150,160,170,180,190,200], "mode": ["uni", "bi"],
           "win_size": 5}

configs = [config1]

bpm_dict = {
    "bpm_avg_x": 0, "bpm_avg_y": 1, "bpm_avg_xy": 2, "bpm_mode_x": 3,
    "bpm_mode_y": 4, "bpm_mode_xy": 5, "bpm_median_x": 6, "bpm_median_y": 7,
    "bpm_median_xy": 8
}

# Load genre mappings
with open("genreID_count_mapping.json", "r") as file:
    genre_Tcount = json.load(file)

with open("genre_symbols_mapping.json", "r") as file:
    genre_name = json.load(file)

tol_type = "rel"
tol = 6
case = "tempo_cases"
foot1s_name = "right_ankle"

for cfg in configs:
    a = cfg["a"]
    w_sec = cfg['win_size']
    h_sec = w_sec/2
    
    for mode in cfg["mode"]:
        for b in cfg['b']:
            foot_1S_df = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/vel/foot/{foot1s_name}_W{w_sec}H{h_sec}_zero_{mode}_{a}_{b}.csv")
            total = foot_1S_df.shape[0]         # total number of 

            json_hits_data = {}
            
            for key in bpm_dict.keys():

                left_hand_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/vel/score/hand/left_wrist_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")
                right_hand_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/vel/score/hand/right_wrist_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")
                
                left_ankle_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/vel/score/foot/left_ankle_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")
                right_ankle_score = pd.read_csv(f"./saved_results/{case}/tempo_{a}_{b}/vel/score/foot/right_ankle_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}_score.csv")

                left_hand_score["hits_idx"] = left_hand_score["hits_idx"].apply(ast.literal_eval)   # convert string to list
                right_hand_score["hits_idx"] = right_hand_score["hits_idx"].apply(ast.literal_eval)

                left_ankle_score["hits_idx"] = left_ankle_score["hits_idx"].apply(ast.literal_eval)
                right_ankle_score["hits_idx"] = right_ankle_score["hits_idx"].apply(ast.literal_eval)

                Lh_hits_idx = set(left_hand_score.iloc[bpm_dict[key]]["hits_idx"])
                Rh_hits_idx = set(right_hand_score.iloc[bpm_dict[key]]["hits_idx"])

                Lf_hits_idx = set(left_ankle_score.iloc[bpm_dict[key]]["hits_idx"])
                Rf_hits_idx = set(right_ankle_score.iloc[bpm_dict[key]]["hits_idx"])

                all_hits_idx = set().union(*[Lh_hits_idx, Rh_hits_idx, Lf_hits_idx, Rf_hits_idx])

                # hits statistics
                hits_stats = {"hits_summary": {"segment": ["Left Hand", "Right Hand", "Left Foot", "Right Foot", "Combined"],
                                            "hits_idx": [Lh_hits_idx, Rh_hits_idx, Lf_hits_idx, Rf_hits_idx, all_hits_idx],
                                            "hits_count": [len(Lh_hits_idx), len(Rh_hits_idx), len(Lf_hits_idx), len(Rf_hits_idx), len(all_hits_idx)],
                                            "total": [total, total, total, total, total],
                                            "hits_percentage": [len(Lh_hits_idx) / total * 100, len(Rh_hits_idx) / total * 100,
                                                                len(Lf_hits_idx) / total * 100, len(Rf_hits_idx) / total * 100,
                                                                len(all_hits_idx) / total * 100]},
                            
                            "hit_idx_by_limb": {"Left Hand": Lh_hits_idx, "Right Hand": Rh_hits_idx,
                                                "Left Foot": Lf_hits_idx,"Right Foot": Rf_hits_idx,
                                                "Combined": all_hits_idx},
                            }
                json_hits_data[key] = hits_stats
                
            save_path1 = f"./saved_results/{case}/tempo_{a}_{b}/stats_vel/{mode}/hit_stats_W{w_sec}H{h_sec}_{tol_type}{tol}_{mode}_{a}_{b}.pkl"

            with open(save_path1, "wb") as f:
                pickle.dump(json_hits_data, f)
                